# Extract tracks matching STARS

Tracking experiment uses 
* ERA5
* control run
* full period (2000-2018)
* Blender & Schubert (2000) matching method with beta = 100

STARS tracks are selected by
* lifetime exceeding 6 h
* within region of 21W, 51E, 64S, 86N

In [1]:
import pandas as pd

from octant.core import TrackRun
from octant.parts import TrackSettings

from common_defs import period
from match_to_stars import _make_match_label
import mypaths

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
dset = "era5"
run_id = 0
match_option = {"method": "bs2000", "beta": 100.0}

## Use previously calculated matches

In [4]:
df = pd.read_csv(
    mypaths.procdir
    / "matches"
    / f"{dset}_run{run_id:03d}_{period}_{_make_match_label(match_option)}.txt",
    skiprows=4,
    names=["pmc", "stars"],
)

## Load the control tracking run

In [5]:
TR = TrackRun.from_archive(mypaths.procdir / f"{dset}_run{run_id:03d}_{period}.h5")

Create an empty `TrackRun` to store a subset of tracks.

In [6]:
tr_sub = TrackRun()

Subset the original data.

In [7]:
tr_sub.data = TR.data.loc[df.pmc]

Create a dummy DataFrame with STARS indices.

In [8]:
stars_inds = pd.DataFrame(
    data = df.stars.values,
    index=tr_sub.data.index.get_level_values(0).unique(),
    columns=["STARS_N"],
)

And store it in an extra column.

In [9]:
tr_sub.data = tr_sub.data.join(stars_inds)
tr_sub.columns.append("STARS_N")

Save the PMCTRACK settings with some comments.

In [10]:
conf_tr = TR.conf.to_dict()
conf_tr.update(
    comment="ERA5 PMC tracks matched to STARS tracks (>=6h, within [21W, 51E, 64S, 86N]) using Blender and Schubert (2000) method with beta=100.",
    author="Denis Sergeev",
    email="",
)

In [11]:
tr_sub.conf = TrackSettings()
tr_sub.conf = tr_sub.conf.from_dict(conf_tr)

## Save the result

In [12]:
fname = mypaths.procdir / "era5_run000_2000_2018__matched_to_stars_6h__bs2000_beta100.h5"

In [13]:
tr_sub.to_archive(fname)

## Test the output

Check that the HDF file can be opened without `octant` and using only `pandas`.

In [14]:
from octant.params import ARCH_KEY

In [15]:
with pd.HDFStore(fname, mode="r") as store:
    df = store[ARCH_KEY]
    metadata = store.get_storer(ARCH_KEY).attrs.metadata
df = df.set_index(TR._mux_names)

In [16]:
df

lon    lat        vo                time        area  \
track_idx row_idx                                                           
7559      0       -17.00  68.50  0.000214 2002-01-10 13:00:00  2283.31470   
          1       -16.75  68.50  0.000276 2002-01-10 14:00:00  3409.24878   
          2       -16.50  68.50  0.000284 2002-01-10 15:00:00  3680.51587   
          3       -15.75  68.75  0.000308 2002-01-10 16:00:00  3658.48828   
          4       -15.50  68.75  0.000305 2002-01-10 17:00:00  4491.12256   
...                  ...    ...       ...                 ...         ...   
59088     28       49.00  70.50  0.000287 2011-04-01 01:00:00  4643.83398   
          29       49.25  70.50  0.000262 2011-04-01 02:00:00  3379.89575   
          30       49.75  70.50  0.000225 2011-04-01 03:00:00  2862.84985   
          31       49.75  70.25  0.000216 2011-04-01 04:00:00  1563.86938   
          32       50.00  70.25  0.000217 2011-04-01 05:00:00   785.11761   

                  vortex_type         slp  STARS_N  
track_idx row_idx                                   
7559      0                 0   983.11115        1  
          1                 0   983.60498        1  
          2                 0   983.73743        1  
          3                 0   984.05365        1  
          4                 0   984.67206        1  
...                       ...         ...      ...  
59088     28                0  1007.13043      140  
          29                0  1007.88544      140  
          30                0  1008.10803      140  
          31                0  1008.79218      140  
          32                0  1009.18927      140  

[3672 rows x 8 columns]